# Diffusion Model for Open Dialogue

Simplified version

References
- DiffuSeq (cited below)

@inproceedings{gong2022diffuseq,
  author = {Gong, Shansan and Li, Mukai and Feng, Jiangtao and Wu, Zhiyong and Kong, Lingpeng},
  booktitle = {International Conference on Learning Representations, ICLR},
  title = {{DiffuSeq}: Sequence to Sequence Text Generation with Diffusion Models},
  year = 2023
}

@article{gong2023diffuseqv2,
  title={DiffuSeq-v2: Bridging Discrete and Continuous Text Spaces for Accelerated Seq2Seq Diffusion Models},
  author={Gong, Shansan and Li, Mukai and Feng, Jiangtao and Wu, Zhiyong and Kong, Lingpeng},
  journal={arXiv preprint arXiv:2310.05793},
  year={2023}
}


## Dataset

- word embeddings (to be loaded?)
- use a corpus

in diffuseq text_datasets.py some steps to load the dataset itself

- [ ] prepare datasets for training and validation in the format (stored as jsonl file?)
```
{"src": "", "train": ""}
```

We should use BERT?

## Training

Note that, in DiffuSeq, a model file is created to store all training progress, configuration etc. (in bash format poitning to raw files?)

- denoise rate ?
- using updates in v2 diffuseq took it from 2 days -> 11 hr learning time

### Loading training data
What are we trying to achieve in summary?

In [ ]:
# train.sh in diffuseq
#  python -m torch.distributed.launch --nproc_per_node=4 --master_port=12233 --use_env run_train.py \
# --diff_steps 2000 \
# --lr 0.0001 \
# --learning_steps 50000 \
# --save_interval 10000 \
# --seed 102 \
# --noise_schedule sqrt \
# --hidden_dim 128 \
# --bsz 2048 \
# --dataset test \
# --data_dir '/Users/sallykalumba/dev/diffusion-dialogue/DiffuSeq/datasets/CommonsenseConversation' \
# --vocab bert \
# --seq_len 128 \
# --schedule_sampler lossaware \
# --notes test-qqp

# saves model & runs tarin.py with arguments

# TODO run on GPU

# train.py 
# parses arguments



In [ ]:
# load the data
# in diffuseq:
# uses a special helper function for loading text data

# data = load_data_text(
#         batch_size=args.batch_size,
#         seq_len=args.seq_len,
#         data_args = args,
#         loaded_vocab=tokenizer,
#         model_emb=model_weight # use model's weights as init
#     )
#     next(data)

#     data_valid = load_data_text(
#         batch_size=args.batch_size,
#         seq_len=args.seq_len,
#         data_args=args,
#         split='valid',
#         deterministic=True,
#         loaded_vocab=tokenizer,
#         model_emb=model_weight # using the same embedding wight with tranining data
#     )


# prints the size of vocab 
# then creates model & diffusion

Chosen dataset must be placed into the below format,

In [ ]:
# Load training data
# referring to load_data_text

"""
For a dataset, create a generator over (seqs, kwargs) pairs.

Each seq is an (bsz, len, h) float tensor, and the kwargs dict contains zero or
more keys, each of which map to a batched Tensor of their own.
The kwargs dict can be used for some meta information.

:param batch_size: the batch size of each returned pair.
:param seq_len: the max sequence length (one-side).
:param deterministic: if True, yield results in a deterministic order.
:param data_args: including dataset directory, num of dataset, basic settings, etc.
:param model_emb: loaded word embeddings.
:param loaded_vocab: loaded word vocabs.
:param loop: loop to get batch data or not.
"""

# load text data


# training_data = get_corpus(data_args, seq_len, split=split, loaded_vocab=loaded_vocab)

# return an iterative data loader

# get a tokenized text data from the training data set 


# if split == 'train':
#         print('### Loading form the TRAIN set...')
#         path = f'{data_args.data_dir}/train.jsonl'
#     elif split == 'valid':
#         print('### Loading form the VALID set...')
#         path = f'{data_args.data_dir}/valid.jsonl'
#     elif split == 'test':
#         print('### Loading form the TEST set...')
#         path = f'{data_args.data_dir}/test.jsonl'
#     else:
#         assert False, "invalid split for dataset"

In [ ]:
import json

data_dir = "./datasets/sample"
path = f'{data_dir}/train.jsonl'

In [ ]:
# read in the data in training data json file 
# TODO do this in a different way 
# FIXME what is the .strip() used for ?
sentence_lst = {'src':[], 'trg': []}
with open(path, 'r') as f_reader:
        for row in f_reader:
            content = json.loads(row)
            sentence_lst['src'].append(content['src'].strip())
            sentence_lst['trg'].append(content['trg'].strip())

In [ ]:
# TODO tokenize datasets

# TODO check the literature to see what the actual steps we are doing here


# get tokenizer
# tokenizer = load_tokenizer(args)

#  loaded_vocab=tokenizer,


# def tokenize_function(examples):
#         input_id_x = vocab_dict.encode_token(examples['src'])
#         input_id_y = vocab_dict.encode_token(examples['trg'])
#         result_dict = {'input_id_x': input_id_x, 'input_id_y': input_id_y}

#         return result_dict




# FIXME where to get the loaded vocab?

# TODO get a dataseet from dict

# FIXME vocab dict is loaded from file 



#     tokenized_datasets = raw_datasets.map(
#         tokenize_function,
#         batched=True,
#         num_proc=4,
#         remove_columns=['src', 'trg'],
#         load_from_cache_file=True,
#         desc="Running tokenizer on dataset",
#     )

In [1]:
# Diffusion-LM (Li et al., 2022)
# embedding function $$EMB(w)$$ to map the discrete text $$w$$ into a continuous space. -->

# TODO make a new tokenizer

"""
    Load tokenizer from bert config or defined BPE vocab dict
    """

# TODO explore using other tokenizer, for this simplified model, just use BERT
# in DiffuSq have the option of a custom tokenizer, and the chosne tokenizer is saved

# TODO what is the difference between AutoTokenizer and BertTokenizer

# tokenizer = AutoTokenizer.from_pretrained(args.config_name)
#             self.tokenizer = tokenizer
#             self.sep_token_id = tokenizer.sep_token_id
#             self.pad_token_id = tokenizer.pad_token_id
#             # save
#             tokenizer.save_pretrained(args.checkpoint_path)


# import torch
# from transformers import BertTokenizer, BertModel, BertConfig

# # Load pre-trained tokenizer and model

# # Bert == loaded vocab
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')


/Users/sallykalumba/dev/diffusion-dialogue/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/sallykalumba/dev/diffusion-dialogue/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
